# The naive Bayes classifier

## Introduction

A supervised classification method, with a lot of assumptions ..., but well adapted to high dimensional data with moderate sample size. 

As reminder:
- logistic regression puts a model on $P(y|x)$ and estimates the parameters by minimizing the cross-entropy loss
- SVM puts a model only on the classification boundary and research the separating hyperplane with largest margin 

Another philosophy is to consider a generative model on data distribution, i.e. model $P(x,y)$ by writting $P(x,y) = P(y)P(x|y)$. Thus it is only needed to estimate $P(x|y)$ for each possible value of $y$ (multivariate gaussian distribution, multivariate multinomial distribution, ...), and also $P(y)$ the prior probabilities (typically simply the proportion of each class). Then the posterior probability $P(y|x)$ can be computed by using the **Bayes theorem**: 

$$
P(y|x) = \frac{P(y)P(x|y)}{P(x)} = \frac{P(y)P(x|y)}{\sum_{k=1}^{K} P(y=k) P(x|y=k)}
$$


## The model

Let assume that $x \in \mathcal{X}$ : 
- **Continuous features**: $\mathcal{X}=\mathcal{R}^d$, $P(x|y=k)$ is modeled by a multivariate density.
- **Categorical features**: $\mathcal{X}=\prod_{j=1}^{d}\{1, ..., m_j\}$, $P(x|y=k)$ is modeled by a multivariate probability.
- Also possible **mixed data**: both continuous and categorical

**Question**: What model use for $P(x|y=k)$?

**Some ideas**:
- Multivariate normal distribution (need only to estimate the vector of expectations and the variance covariance matrix) 
- Mixture of multivariate normal distribution for more flexibility
- Not so many models for categorical data ... 

**The Naives assumtion**:
$$
P(x|y=k) = P(x_1, \ldots, x_d |y=k) = \prod_{j=1}^{d} P(x_j|y=k),
$$ 

It assumes that all the variables are independant given the class. 

- Avoid to model dependency! 
- Only need to model $P(x_j|y=k)$ by a univariate distribution
    - univariate discrete probability distribution for categorical variables (multinomial, Poisson, ...) 
    - univariate probability density function (normal, Student, Gamma, exponential, ...)


Let assume that $P(x_j|y=k)$ belong to some parametric family we will denote  $P(x_j|y=k) := P(x_j|\theta_{kj})$  where $\theta_{kj}$ are the paramters of the distribution of variable $j$ in class $k$:
- $\theta_{kj} = (\mu_{kj} , \sigma_{kj}^2)$ for a Gaussian distribution
- $\theta_{kj} = (\alpha_{kj1}, ..., \alpha_{kjm_{j}})$ for a multinomial distribution where $\alpha_{kjh}$ is the probability of the model $h$ of variable $j$ in class $k$


## Parameters estimation

Parameters are estimated by maximum likelihood as in statistical inference!

Let $\theta_k = (\theta_{k1}, ..., \theta_{kd})$ that groups all the parameter of class $k$, let also denote by $\pi_k = P(y=k)$ the prior probability of class which also need to be estimated. 

Let denote by $\theta = (\pi_1, ..., \pi_K, \theta_1, \ldots, \theta_K)$ that groups all the paramters of the model 

The likelihood is: 
$$
\ell(\theta) = \sum_{k=1}^{K} \sum_{i \in \mathcal{C}_k} \pi_k \log P(x_i | \theta_k)
$$
with $\mathcal{C}_k$ the set of data points belonging to class $k$


**Estimation of the prior probabilities**

For all $k \in \{1, ..., K \}$
$$
\hat \pi_k = \frac{n_k}{n}
$$
with $n_k$ the number of data in class $k$, among the total of $n$ data. 

*Remark*: This can depend on the sampling scheme, for instance in the medical setting we can consider restrospective sampling, i.e. fix by advance the proportion of each class in the training data, for instance for rare diseases have the same number of patient with and without the disease in the cohort. In this case the user can give manually the proportion of each class in the whole population.  


**Gaussian distribution**
For $k \in \{1, ..., K\}$, $j \in \{1,...,d\}$
$$
\hat\mu_{kj} = \frac{\sum_{i \in \mathcal{C}_k} x_{ij}}{n_k}
$$

$$
\hat\sigma_{kj}^2 = \frac{\sum_{i \in \mathcal{C}_k} (x_{ij}-\hat\mu_{kj})^2}{n_k}
$$


**Multinomial distribution**
For $k \in \{1, ..., K\}$, $j \in \{1,...,d\}$ and $h \in \{1,...,m_j\}$
$$
\hat\alpha_{kjh} = \frac{n_{kjh}}{n_k}
$$
with $n_{kjh}$ the number of times that modality $h$ of variable $j$ has been observed in class $k$.

*Remark*: Regularized version of the estimators can be consider to avoid to estimate some $\hat\alpha_{kjh}$ to $0$. For instance $\hat\alpha_{kjh} = \frac{n_{kjh} + c}{n_k + c m_j}$ where $c>0$ is some regularization parameter.


## Prediction of the class for new data

Based on $\hat\theta$, it is possible to compute $\forall k \in \{1, ..., K\}$:
$$
P(y=k|x, \hat\theta) = \frac{\hat\pi_k P(x|\hat\theta_k)}{\sum_{k=1}^{K} \hat\pi_k P(x|\hat\theta_k)}
$$

Then the predicted class can be obtained by Maximum A Posteriori (MAP)

$$
\hat y = \arg\max_{k \in \{1, ..., K \}} P(y=k|x, \hat\theta)
$$

 
## Discussion 

The model make stong assumptions, i.e. models the distribution of $(x,y)$, where only the distribution of $y|x$ or even only the decision boundary is needed to make prediction. 

Thus this leds to model miss-specification which may degrade the performance of the final classifier. However such kind of model can still work well with moderate sample size and high number of variables. See for instance [1]

[1] Hand, D. J., & Yu, K. (2001). Idiot's Bayes—not so stupid after all?. International statistical review, 69(3), 385-398.





# 1. Naive Bayes in sklearn

More information can be found on :

https://scikit-learn.org/stable/modules/naive_bayes.html

The main functions are :
- `GaussianNB` : Naive Bayes for continuous features using Gaussian assumption
- `BernoulliNB` : Naive Bayes classifier for multivariate Bernoulli models.
- `CategoricalNB` : Naive Bayes classifier for categorical features.
- `MultinomialNB` : Naive Bayes classifier for multinomial models.
- `ComplementNB` : Complement Naive Bayes classifier.

For mixed-type of features there is not dedicated function in sklearn, but continous features can for instances be discretized in order to use `ComplementNB` on the discretized dataset.

**Q1** : By looking at the documentation explain the differences between `BernoulliNB`, `CategoricalNB` and `MultinomialNB`

## Continuous features

In [ ]:
%matplotlib inline
#necessary imports
import time
import matplotlib.pyplot as plt
import numpy as np
from random import randint
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

In [ ]:
# loading the dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

**Q2**: Apply the `GaussianNB` to train the model on the iris dataset

**Q3**: Let consider the following functions
- `predict_joint_log_proba(X)`
- `predict_log_proba(X)`
- `predict_proba(X)`

(a) Explain what the above functions do and illustrate it on the iris dataset

(b) Explain the use of making use of the log instead of directly computing the probability

**Q3bis** : Have a look at some basic implementation of the Naive Bayes algorithm 

https://www.python-engineer.com/courses/mlfromscratch/05_naivebayes/

(a) Complete the code in order to compute the posterior membership probabilities (and not only the class with the highest posterior probability)

(b) Also consider directly compute the log of the pdf rather than the pdf for numerical stability

(c) What modification should you consider to consider categorical features?

(d) Give some ideas of how it would be possible to consider mixed type of data. 

## Categorical features

**Q4**: Use `KBinsDiscretizer` of `sklearn.preprocessing` in order to discrtize the iris data with `n_bins=2`, then train a naive bayes classifier in this discretized dataset

**Q5**: Idem but with `n_bins = 3`

**Q6**: Compare the performances of the different approches

## Application on the MNIST dataset

**Q7**: Apply the Naive Bayes classifier on the MNIST dataset and comment its performances

## 2. Linear and quadratic discriminant analysis

The model now considered is not the naives Bayes, since it does not any more the assumption of conditional independence. It is however still a generative classifier since it models the distribution of $(x,y)$ then it deduce the distribution of $y|x$ using the Bayes theorem. 

More information can be found on : 

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html

The **linear discriminant analysis** assumes multivariate Gaussian distribution in each class, and moreover the covariance matrices are assumed to be the same in each class: 
$$
x | y = k \sim \mathcal{N}(\mu_k , \Sigma)
$$
This results in a linear decision boundary. 


The **quadratic discriminant analysis** assumes multivariate Gaussian distribution in each class, and moreover the covariance matrices are assumed to be the same in each class: 
$$
x | y = k \sim \mathcal{N}(\mu_k , \Sigma_k).
$$
This results in a quadratic decision boundary.

**Remarks** : 

- Linear discriminant analysis can also be used for dimension reduction by looking for the most discriminant components (K-1 discriminant components if $K$ classes are considered) 
- In high dimensional setting the `shrinkage` parameter can be used in order to regularize the estimator of the covariance matrix, for example of the kind $(1-\alpha)\hat\Sigma + \alpha I$



**Q8** : Fit a linear discriminant analysis on the MNIST dataset, try to optimize the value of the shrinkage parameter